# Kaggle Challenge - Learning Equality
https://www.kaggle.com/competitions/learning-equality-curriculum-recommendations/overview

## Goal of the Competition

The National Football League (NFL) is back with another Big Data Bowl, where contestants use Next Gen Stats player tracking data to generate actionable, creative, and novel stats. Previous iterations have considered running backs, defensive backs, and special teams, and have generated metrics that have been used on television and by NFL teams. In this year’s competition, you’ll have more subtle performances to consider—and potentially more players to measure.

## Submission File

For each **topic_id** in the test set, you must predict a space-delimited list of recommended **content_ids** for that topic. The file should contain a header and have the following format:

~~~
topic_id,content_ids
t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c_76231f9d0b5e
t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c_ebb7fdf10a7e
t_00069b63a70a,c_11a1dc0bfb99
...
~~~

## Scoring
Mean F2 score

# Evaluation Metric - Efficiency Scoring
We compute a submission's efficiency score by:

\begin{equation} \text{Efficiency} = \frac{1}{ \text{Benchmark} - \max\text{F2} }\text{F2} + \frac{1}{32400}\text{RuntimeSeconds} \end{equation}


where **F2** is the submission's score on the main competition metric, **Benchmark** is the score of the benchmark sample_submission.csv, **maxF2** is the maximum  of all submissions on the Private Leaderboard, and **RuntimeSeconds** is the number of seconds it takes for the submission to be evaluated. The objective is to minimize the efficiency score.

During the training period of the competition, you may see a leaderboard for the public test data in the following notebook, updated daily: Efficiency Leaderboard. After the competition ends, we will update this leaderboard with efficiency scores on the private data. During the training period, this leaderboard will show only the rank of each team, but not the complete score.

# Data

## Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
from sklearn.dummy import DummyClassifier
from sklearn.multioutput import MultiOutputClassifier

In [3]:
# Download files with API
#!kaggle competitions download -c learning-equality-curriculum-recommendations

# Data Collection

In [4]:
# load the data into pandas dataframes
df_topics = pd.read_csv("topics.csv").fillna({"title": "", "description": ""})
df_content = pd.read_csv("content.csv").fillna("")
df_corr = pd.read_csv("correlations.csv")
df_topics

,id,title,description,channel,category,level,language,parent,has_content
0,t_00004da3a1b2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True
1,t_000095e03056,Unit 3.3 Enlargements and Similarities,,b3f329,aligned,2,en,t_aa32fb6252dc,False
2,t_00068291e9a4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True
3,t_00069b63a70a,Transcripts,,6e3ba4,source,3,en,t_4054df11a74e,True
4,t_0006d41a73a8,Графики на експоненциални функции (Алгебра 2 н...,Научи повече за графиките на сложните показате...,000cf7,source,4,bg,t_e2452e21d252,True
...,...,...,...,...,...,...,...,...,...
76967,t_fffb0bf2801d,4.3 Graph of functions,,e77b55,aligned,4,en,t_676e6a1a4dc7,False
76968,t_fffbe1d5d43c,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True
76969,t_fffe14f1be1e,Lección 7,,6e90a7,aligned,6,es,t_d448c707984d,True
76970,t_fffe811a6da9,تحديد العلاقة بين الإحداثيّات القطبية والإحداث...,5b9e5ca86571f90499ea987f,9fd860,source,2,ar,t_5b4f3ba4eb7d,True


In [5]:
df_content

,id,title,description,kind,text,language,copyright_holder,license
0,c_00002381196d,"Sumar números de varios dígitos: 48,029+233,930","Suma 48,029+233,930 mediante el algoritmo está...",video,,es,,
1,c_000087304a9e,Trovare i fattori di un numero,Sal trova i fattori di 120.\n\n,video,,it,,
2,c_0000ad142ddb,Sumar curvas de demanda,Cómo añadir curvas de demanda\n\n,video,,es,,
3,c_0000c03adc8d,Nado de aproximação,Neste vídeo você vai aprender o nado de aproxi...,document,\nNado de aproximação\nSaber nadar nas ondas ...,pt,Sikana Education,CC BY-NC-ND
4,c_00016694ea2a,geometry-m3-topic-a-overview.pdf,geometry-m3-topic-a-overview.pdf,document,Estándares Comunes del Estado de Nueva York\n\...,es,Engage NY,CC BY-NC-SA
...,...,...,...,...,...,...,...,...
154042,c_fffcbdd4de8b,2. 12: Diffusion,,html5,What will eventually happen to these dyes?\n\n...,en,CSU and Merlot,CC BY-NC-SA
154043,c_fffe15a2d069,Sommare facendo gruppi da 10,Sal somma 5+68 spezzando il 5 in un 2 e un 3.\n\n,video,,it,,
154044,c_fffed7b0d13a,Introdução à subtração,Sal fala sobre o que significa subtrair. Os ex...,video,,pt,,
154045,c_ffff04ba7ac7,SA of a Cone,,video,,en,,


In [6]:
df_corr

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d c_376c5a8eb028 c_5bc0e1e2cba0 c...
1,t_00068291e9a4,c_639ea2ef9c95 c_89ce9367be10 c_ac1672cdcd2c c...
2,t_00069b63a70a,c_11a1dc0bfb99
3,t_0006d41a73a8,c_0c6473c3480d c_1c57a1316568 c_5e375cf14c47 c...
4,t_0008768bdee6,c_34e1424229b4 c_7d1a964d66d5 c_aab93ee667f4
...,...,...
61512,t_fff830472691,c_61fb63326e5d c_8f224e321c87
61513,t_fff9e5407d13,c_026db653a269 c_0fb048a6412c c_20de77522603 c...
61514,t_fffbe1d5d43c,c_46f852a49c08 c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [7]:
df_corr['content_ids'] = df_corr.content_ids.str.split(' ')
y_train = df_corr.explode('content_ids')
y_train

,topic_id,content_ids
0,t_00004da3a1b2,c_1108dd0c7a5d
0,t_00004da3a1b2,c_376c5a8eb028
0,t_00004da3a1b2,c_5bc0e1e2cba0
0,t_00004da3a1b2,c_76231f9d0b5e
1,t_00068291e9a4,c_639ea2ef9c95
...,...,...
61513,t_fff9e5407d13,c_d64037a72376
61514,t_fffbe1d5d43c,c_46f852a49c08
61514,t_fffbe1d5d43c,c_6659207b25d5
61515,t_fffe14f1be1e,c_cece166bad6a


In [8]:
y_train.topic_id.value_counts()
y_train['topic_id']

0        t_00004da3a1b2
0        t_00004da3a1b2
0        t_00004da3a1b2
0        t_00004da3a1b2
1        t_00068291e9a4
              ...      
61513    t_fff9e5407d13
61514    t_fffbe1d5d43c
61514    t_fffbe1d5d43c
61515    t_fffe14f1be1e
61516    t_fffe811a6da9
Name: topic_id, Length: 279919, dtype: object

In [9]:
df_X = y_train
df_X = df_X.merge (df_topics.rename(columns={'id': 'topic_id'}), on='topic_id', how='left')
df_X = df_X.merge (df_content.rename(columns={'id': 'content_ids'}), on='content_ids', how='left')
df_X.rename(columns={'title_x': 'title_topic', 'title_y': 'title_content', 'description_x': 'description_topic', 'description_y': 'description_content'}, inplace = True)
df_X
X_train = df_X.drop(columns = ['topic_id', 'content_ids'])
X_train

,title_topic,description_topic,channel,category,level,language_x,parent,has_content,title_content,description_content,kind,text,language_y,copyright_holder,license
0,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Молив като резистор,"Моливът причинява промяна в отклонението, подо...",video,,bg,,
1,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Да чуем променливото съпротивление,Тук чертаем линия на лист хартия и я използвам...,video,,bg,,
2,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Променлив резистор (реостат) с графит от молив,Използваме сърцевината на молива (неговия граф...,video,,bg,,
3,Откриването на резисторите,"Изследване на материали, които предизвикват на...",000cf7,source,4,bg,t_16e29365b50d,True,Последователно свързване на галваничен елемент...,"Защо отклонението се променя, когато се свърже...",video,,bg,,
4,Entradas e saídas de uma função,Entenda um pouco mais sobre funções.,8e286a,source,4,pt,t_d14b6c2a2b70,True,Dados e resultados de funções: gráficos,Encontre todas as entradas que correspondem a ...,exercise,,pt,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279914,NA_U06 - El periódico,,71fd51,supplemental,2,es,t_5bd8f6ae9f7d,True,Introducción: El periódico,,html5,,es,,
279915,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True,Proof: Right triangles inscribed in circles -d...,Proof showing that a triangle inscribed in a c...,video,,sw,,
279916,Inscribed shapes problem solving,Use properties of inscribed angles to prove pr...,0c929f,source,4,sw,t_50145b9bab3f,True,Area of inscribed equilateral triangle -dubbed...,A worked example of finding the area of an equ...,video,,sw,,
279917,Lección 7,,6e90a7,aligned,6,es,t_d448c707984d,True,Juego con las palabras,,document,,es,,


# Model

## Preprocessing

## Baseline Score

In [ ]:
dummy = DummyClassifier (strategy = 'most_frequent')
multi_target_dummy = MultiOutputClassifier(dummy, n_jobs=2)
multi_target_dummy.fit(X_train, y_train)

In [ ]:
multi_target_dummy.predict (X_train)

In [58]:
# Calculate the f2-score
y_train = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
F2 = fbeta_score(y_train, y_pred, beta=2)

In [74]:
X_ex = X_train[17:22]
y_ex = y_train[17:22]
X_ex

,title_topic,description_topic,channel,category,level,language_x,parent,has_content,title_content,description_content,kind,text,language_y,copyright_holder,license
17,12. 20: Bird Reproduction,NaN,ebc86c,supplemental,5,en,t_c44ac9711007,True,12. 20: Bird Reproduction,NaN,html5,Is this pair of birds actually a “couple”?\n\n...,en,CSU and Merlot,CC BY-NC-SA
18,12. 20: Bird Reproduction,NaN,ebc86c,supplemental,5,en,t_c44ac9711007,True,Astounding Mating Dance Birds of Paradise -- H...,The Birds of Paradise from BBC's outstanding P...,video,NaN,en,NaN,NaN
19,2.1.2 - Logarithms,NaN,e77b55,aligned,5,en,t_b897d168db90,True,Proof of the logarithm change of base rule,Sal proves the logarithmic change of base rule...,video,What I want to do in this video is prove the c...,en,Khan Academy,CC BY-NC-SA
20,2.1.2 - Logarithms,NaN,e77b55,aligned,5,en,t_b897d168db90,True,Intro to logarithm properties (2 of 2),Sal introduces the logarithm identities for mu...,video,PROFESSOR: Welcome back. I'm going to show you...,en,Khan Academy,CC BY-NC-SA
21,2.1.2 - Logarithms,NaN,e77b55,aligned,5,en,t_b897d168db90,True,Solve exponential equations using logarithms: ...,Solve exponential equations that have 2 or oth...,exercise,NaN,en,NaN,NaN


In [75]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder

model = RandomForestClassifier(random_state=1)
multi_target_model = MultiOutputClassifier(model, n_jobs=-1)
multi_target_model.fit(X_ex, y_ex)
y_pred = multi_target_model.predict(X_ex)

ValueError: could not convert string to float: '12. 20: Bird Reproduction'